In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from google.cloud import bigquery
from bq_carrefour import MethodBQ
import numpy as np

In [ ]:
'''
Funcion para normalizar el padron y dejarlo Limpio y operativo
'''
### COMIENZO A TRABAJAR SOBRE EL PADRON ###
    # Cargo la Informacion del padron
try:
    # Indico que columnas voy a necesitar
    cols = ['GSX', 'NOMBRE', 'Fecha apertura', 'FIN DE CIERRE','ORGANIZACIÓN ', 'DIRECTOR EXPLOTACIÓN', 'DIRECTOR OPERACIONAL', 'DIRECTOR / GERENTE REGIONAL', 'SUB REGION', 'DIRECTOR/ GERENTE TIENDA', 'Provincia Tableau', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'CAJAS', 'COD.POSTAL']
    pad = pd.read_excel('data/padron.xlsx', header=17, usecols=cols)

except Exception as e:
    print('Error a la hora de cargar el Padron. Detalle {e}')

# Estandarizo un poco los nombres de las columnas
pad.columns = pad.columns.str.strip().str.lower().str.replace(' /', '').str.replace('/', '').str.replace('.', ' ')

# Quito nulos
pad = pad.dropna(subset=['nombre', 'organización', 'fecha apertura'])

# Me quedo unicamente con formatos validos
pad = pad[pad['organización'].isin(['HIPERMERCADO', 'MAXI', 'Market', 'Express', 'E-Commerce'])]

# Genero una columna de Estado
pad['estado'] = np.where(pad['fin de cierre'] == '-', 'Activa', 'Cerrada')

# Quito columna incompleta
pad = pad.drop(columns={'provincia tableau'})

# Renombro la columna completa de provincias
pad = pad.rename(columns={
    'provincia':'provincia tableau',
    'gsx':'id tienda'
})

# Genero un list aux para bucle abajo
cols = ['m² pgc', 'm² pft', 'm² bazar', 'm² electro', 'm² textil', 'm² pls', 'm² galerias', 'cajas', 'm² parcking']

# Itero sobre cols para estandarizar, rellenar y limpiar nulos
for col in cols:
    pad[col] = pad[col].fillna(0).replace('-', 0).replace('sd', 0).replace('SD', 0).replace('', 0).astype(int)

# Ordeno columnas
pad = pad[['nombre', 'id tienda', 'fecha apertura', 'organización', 'director explotación', 'director operacional', 'director gerente regional', 'sub region', 'director gerente tienda', 'provincia tableau', 'm² salón', 'm² pgc', 'm² pft', 'm² bazar', 'm² electro', 'm² textil', 'm² pls', 'm² galerias', 'cod postal', 'cajas', 'm² parcking', 'estado']]

# Convierto el codigo postal a String ya que es alfanumerico
pad['cod postal'] = pad['cod postal'].astype(str)

pad['fecha apertura'] = pd.to_datetime(pad['fecha apertura'], format='%Y/%m/%d', errors='coerce')

# Genero una nueva columna para identificar la ultima fecha de actualizacion de las tiendas
pad['modificacion'] = datetime.today().strftime('%d/%m/%Y')

# Formateo la fecha y la convierto a String. Como no es util para comprar o utilizar en series de tiempo, sirve igual
pad['modificacion'] = pad['modificacion'].astype('str')

# Reseteo y quito Indice indeseado
pad = pad.reset_index(drop=True)

# Realizo una transformacion para Normalizar valores de las provincias
pad['provincia tableau'] = pad['provincia tableau'].str.strip().str.upper()
pad['provincia tableau'] = np.where(pad['provincia tableau'] == 'NEUQUÉN', 'NEUQUEN', pad['provincia tableau'])

#Remplazo cualquier guion por nans
pad = pad.replace('-', np.nan)

In [84]:
def crear_tabla_si_no_existe(df: pd.DataFrame, table_id: str, project_id: str):
    client = bigquery.Client(project=project_id)

    try:
        client.get_table(table_id)
        print(f"✅ La tabla {table_id} ya existe")
    except NotFound:
        print(f"⚠️ La tabla {table_id} no existe. Creándola...")

        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_EMPTY"
        )

        load_job = client.load_table_from_dataframe(
            dataframe=df,
            destination=table_id,
            job_config=job_config
        )

        load_job.result()
        print(f"✅ Tabla {table_id} creada correctamente.")

In [85]:
def carga_padron(padron, project_id='gcp-ar-cdg-datos-dev', dataset_id='marketshare_project', table_name='padron'):
    try:
        client = bigquery.Client(project=project_id)

        # Construimos correctamente el ID completo de la tabla
        full_table_id = f"{project_id}.{dataset_id}.{table_name}"

        # Paso previo: crear tabla si no existe
        crear_tabla_si_no_existe(padron, full_table_id, project_id)

        # Configuración del job de carga
        job_config = bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition="WRITE_TRUNCATE"  # 👈 Sobrescribir información
        )
        
        # Subir el DataFrame
        job = client.load_table_from_dataframe(padron, full_table_id, job_config=job_config)
        job.result()

        print(f"✅ Éxito al cargar la información del padrón a GCP: {full_table_id}")

    except Exception as e:
        print(f"❌ Error al subir el padrón a GCP: {e}")

In [13]:
import pandas as pd

df = pd.read_excel('data/padron.xlsx', header=17)

In [14]:
df.columns.to_list()

['Unnamed: 0',
 'GSX',
 'BW',
 'BFC entity',
 'BFC format',
 'SAP',
 'TD',
 'Pegasus',
 'PREV NAMES',
 'PREV ALIAS',
 'NOMBRE',
 'Fecha apertura',
 'BANDERA',
 'ORGANIZACIÓN ',
 'DIRECTOR EXPLOTACIÓN',
 'DIRECTOR OPERACIONAL',
 'DIRECTOR / GERENTE REGIONAL',
 'SUB REGION',
 'DIRECTOR/ GERENTE TIENDA',
 'MODELO',
 'Fecha Lanzamiento',
 'CONTROLADOR',
 'latitude',
 'longitude',
 'Provincia Tableau',
 'Provincia Shapefile',
 'Municipalidad Shapefile',
 'M² SALÓN',
 'M² PGC',
 'M² PFT',
 'M² BAZAR',
 'M² Electro',
 'M² Textil',
 'M² Pls',
 'M² GALERIAS',
 'DOMICILIO',
 'COD.POSTAL',
 'CELULAR TIENDA',
 'BARRIO/LOCALIDAD',
 'PROVINCIA',
 'MUNICIPALIDAD',
 'CELULAR RESPONSABLE',
 'PROPIEDAD',
 'PARQUE',
 'FECHA CAMBIO DE BANDERA',
 'EX BANDERA',
 'DIAS CERRADOS',
 'CAJAS',
 'M² Parcking',
 'INICIO DE REFORMA',
 'FIN DE REFORMA',
 'FIN DE CIERRE',
 'Transformaciones',
 's',
 'clase',
 'Unnamed: 55',
 'Unnamed: 56',
 'Unnamed: 57',
 'Unnamed: 58',
 'Unnamed: 59',
 'Unnamed: 60',
 'Unnamed: 61'